In [11]:
import numpy as np

In [12]:
import os
PATH = os.getcwd()
FILE_NAME = "corpus_100k"
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
corpus = PlaintextCorpusReader(PATH,FILE_NAME)

In [13]:
corpus_raw = corpus.raw()
corpus_raw = corpus_raw.lower()

In [26]:
words = []
for word in corpus_raw.split():
    if word != '.' and len(word)!=1 and len(word)!=2: 
        words.append(word)
words = set(words) 
word2int = {}
int2word = {}
vocab_size = len(words)
for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [28]:
words

{'կմիացվեն',
 'կանխակալության',
 'կազմակերպան',
 'լաքեյի',
 'քջում',
 'բարձրախոսն',
 '691,6',
 'կրտսերների',
 'տուգանքը',
 'ըբկ',
 'սլոթեր',
 'հարցաննությունը',
 'օգտատե',
 'բնակելիներ',
 'էկզոկլետեոնով',
 'պոտյոյի',
 '6,96',
 'առաքելությունն',
 'չհաստատելու',
 'ծնվեցին',
 'քեոսայանն',
 '60-նն',
 'տարեսկզբի',
 'տեղացիներուն',
 'տրամադրվածությունն',
 'փոխնախագհ',
 'երթևեկելը',
 'մաչու-պիկչուի',
 'նորեն',
 'չշփոթել',
 'խայթոցը',
 'չթափեց',
 'emui',
 'գոլդեն',
 'հբճ-ի',
 'անցակացվելիք',
 'պաշտոնազրկումները',
 '5.53',
 '1.390',
 'ինքնավերականգնում',
 'լուսնագնացի',
 'թերյելը',
 'ներդնեն',
 'տուրն',
 '275.01',
 'կսմթեց',
 '02.26',
 'մեկնումեկը',
 'տեղաշարժեի',
 'կհավասարեցնենք',
 'business-ին',
 'կաթնաշոռը',
 'մտերմացել',
 'ռբա',
 'երկվա',
 'կալիզիրն',
 'մսատեսակներից',
 'ուրացում',
 'լեն-թոր',
 'ռազմաբազային',
 'պեմզա',
 'խանց',
 'tengrinews-ը',
 'հայկան',
 'ճահիճ',
 'ֆենտոնին',
 'վստահեցնելու',
 'հնարով',
 'ձայնագրած',
 'спальне',
 'զբաղվողին',
 'մատիդ',
 'գրոց',
 'university',
 'սմայլիկն

In [23]:
print(vocab_size)

348800


In [6]:
print(int2word[76369])

բարդացրինք


In [7]:
raw_sentences = corpus_raw.split('.')
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())

In [8]:
len(sentences)

255600

In [9]:
print(sentences[0])

['հիմա', 'ես', 'կարող', 'է', 'մի', 'բառ', 'մի', 'քիչ', 'անզգույշ', 'արտահայտեմ', ',', 'եւ', 'երեք', 'ամիս', 'հհ', 'ներքաղաքական', 'կյանքն', 'այդպես', 'էլ', 'չխաղաղվի']


In [10]:
data = []
WINDOW_SIZE = 2
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

KeyboardInterrupt: 

In [ ]:
print(len(data))

In [ ]:
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size, dtype='uint8')
    temp[data_point_index] = 1
    return temp
x_train = [] # input word
y_train = [] # output word
for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))
# convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

In [ ]:
EMBEDDING_DIM = 5 # you can choose your own number
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
hidden_representation = tf.add(tf.matmul(x,W1), b1)

In [ ]:
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) #make sure you do this!
# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))
# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 10000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))

In [ ]:
print(sess.run(W1))
print('----------')
print(sess.run(b1))
print('----------')

In [ ]:
vectors = sess.run(W1 + b1)


print(vectors)

In [ ]:
print(vectors[ word2int['queen'] ])

In [ ]:
print(int2word[find_closest(word2int['king'], vectors)])
print(int2word[find_closest(word2int['queen'], vectors)])
print(int2word[find_closest(word2int['royal'], vectors)])

In [ ]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
vectors = model.fit_transform(vectors)

In [ ]:
from sklearn import preprocessing
normalizer = preprocessing.Normalizer()
vectors =  normalizer.fit_transform(vectors, 'l2')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
for word in words:
    print(word, vectors[word2int[word]][1])
    ax.annotate(word, (vectors[word2int[word]][0],vectors[word2int[word]][1] ))
plt.show()